In [ ]:
import platform, ctypes, os, requests, shutil
import pandas as pd
import getpass
from ipywidgets import interactive, interact_manual, interact
import shutil

In [ ]:
winver = platform.version().split('.')[2]
winbuild = int(winver)

In [ ]:
def isAdmin():
    try:
        is_admin = (os.getuid() == 0)
    except AttributeError:
        is_admin = ctypes.windll.shell32.IsUserAnAdmin() != 0
    return is_admin

In [ ]:
def install_wsl():
    if isAdmin():
        print("Please restart your system after installation is finished, before going forward.")
        !dism.exe /online /enable-feature /featurename:Microsoft-Windows-Subsystem-Linux /all /norestart
        if winbuild >= 19041:
            #If you have higher version you can update immediately hereafter restart
            !dism.exe /online /enable-feature /featurename:VirtualMachinePlatform /all /norestart
            #Set WSL 2 as your default version
            !wsl --set-default-version 2
    else:
        print("You need to run \'install_wsl\' as Administrator. Please re-open this Jupyter Notebook as Administrator for this part.")

In [ ]:
def update_wsl2():
    if isAdmin():
        url = 'https://wslstorestorage.blob.core.windows.net/wslblob/wsl_update_x64.msi'
        location = %pwd
        if url.find('/'):
            fname = url.rsplit('/', 1)[1]
        flocation = location + '\\' + fname
        if os.path.isfile(flocation):
            print("WSL2 Update file already downloaded")
        else:
            print("Location to save file:", location)
            with requests.get(url, stream=True) as r:
                with open(flocation, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)

        !msiexec /i $flocation /qn #fname
        !wsl --set-default-version 2
    else:
        print("You need to run \'update_wsl2\' as Administrator. Please re-open this Jupyter Notebook as Administrator for this part.")

In [ ]:
wsl_df = pd.read_csv('req\\WSL_VMs.csv') 

In [ ]:
def add_distro(distro):
    if shutil.which(distro) is None:
        print("Trying to install", distro)
        fname = distro + '.appx'
        location = %pwd
        flocation = location + '\\' + fname
        if os.path.isfile(flocation):
            print("WSL VM file already downloaded")
        else:
            url = wsl_df.URL[(wsl_df.Name == distro)].values
            url = ''.join(url)

            print("Location to save file:", location)
            with requests.get(url, stream=True) as r:
                with open(flocation, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)

        !powershell "Add-AppxPackage -Path '{flocation}' -Verbose"
        print("Done")
    else:
        print("Distro already installed.")

In [ ]:
def add_distro(distro):
    try:
        !{distro} install --root
        print(distro, "was already available and now installed.")
    except:
        print("Trying to install", distro)
        fname = distro + '.appx'
        location = %pwd
        flocation = location + '\\' + fname
        if os.path.isfile(flocation):
            print("WSL VM file already downloaded")
        else:
            url = wsl_df.URL[(wsl_df.Name == distro)].values
            url = ''.join(url)

            print("Location to save file:", location)
            with requests.get(url, stream=True) as r:
                with open(flocation, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)

        !powershell "Add-AppxPackage -Path '{flocation}' -Verbose"
        print("Done")

In [ ]:
def installed_distro():
    distro_lst = !wsl --list -q
    distro_lst = [d.replace('\x00', '') for d in distro_lst]
    #Remove empty strings in list
    distro_lst = list(filter(None, distro_lst))
    return(distro_lst)

In [ ]:
def get_installed(Distro):
    return(Distro)

In [ ]:
def install_update_packages(VM_Name):
    !wsl -d {VM_Name} apt-get update
    !wsl -d {VM_Name} apt-get install -y openssl sudo wget
    !wsl -d {VM_Name} apt-get -y upgrade
    !wsl -d {VM_Name} apt-get -y autoremove

https://raccoon.ninja/en/dev/using-python-to-check-if-the-application-is-running-as-an-administrator/

https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests